In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re
import datetime as dt
from bureau_fc import get_bureau_feats
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import copy
from sklearn.metrics import f1_score

In [2]:
import sys
sys.path.append('ml_lib/')
from encoding import FreqeuncyEncoding
from custom_classifier_mutliclass import Estimator
from hyperopt_multiclass import HyperOptModelSelection
from hyperopt import hp
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

In [3]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [4]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')

In [5]:
target_map = {'No Top-up Service': 0,
 '12-18 Months': 1,
 '18-24 Months': 2,
 '24-30 Months': 3,
 '30-36 Months': 4,
 '36-48 Months': 5,
 ' > 48 Months': 6,
 }
train['Top-up Month'] = train['Top-up Month'].map(target_map)

In [6]:
reverse_map = {v:k for k,v in target_map.items()}

In [7]:
reverse_map

{0: 'No Top-up Service',
 1: '12-18 Months',
 2: '18-24 Months',
 3: '24-30 Months',
 4: '30-36 Months',
 5: '36-48 Months',
 6: ' > 48 Months'}

In [8]:
train['Top-up Month'].value_counts()

0    106677
6      8366
5      3656
3      3492
4      3062
2      2368
1      1034
Name: Top-up Month, dtype: int64

In [9]:
df = pd.read_pickle('sahil_top_500_feats.pkl')
df2 = pd.read_pickle('nikhil_feats.pkl')
df2 = df2[df2.columns[~df2.columns.isin(df.columns)].tolist()+['ID']]
df2.shape
df = df.merge(df2,on='ID')
del df2
import gc
gc.collect()

0

In [10]:
df.shape

(143400, 1551)

In [11]:
%%time
cols = df.columns[(df.max()<32000)&(df.min()>-32000)]
for i,col in enumerate(cols):
    df[col] = df[col].astype('float32')
    if i%100==0:
        print(i)

0
100
200
300
400
500
600
700
CPU times: user 57.5 s, sys: 58.1 s, total: 1min 55s
Wall time: 1min 55s


In [12]:
gc.collect()

22

In [13]:
cat_cols=[]
target = 'Top-up Month'
drop_cols = ['ID','DisbursalDate','MaturityDAte','AuthDate','AssetID']
num_cols = (df.columns[~df.columns.isin([target]+drop_cols+cat_cols)].tolist())
use_cols = cat_cols+num_cols

In [14]:
len(num_cols)

1549

In [15]:
train = df[df.ID.isin(train.ID)]
test = df[df.ID.isin(test.ID)]
train = train.sort_values('ID',ascending=True)
test = test.sort_values('ID',ascending=True)
train[target] = train[target].astype('int')

In [16]:
train.shape,test.shape

((128655, 1551), (14745, 1551))

In [18]:
del df,
gc.collect()

22

In [17]:
# train.head()

In [18]:
# use_cols = ['MonthlyIncome', 'cal_tenor', 'State', 'emi_sal_ratio', 'AGE', 'City', 'SupplierID', 'BranchID', 'pin3', 'pin1', 'disb_year', 'EMI', 'pin2', 'AssetCost', 'LTV', 'min_tenor', 'ltv2', 'median_tenor', 'max_tenor', 'ltfs_min_tenor', 'ltfs_max_tenor', 'disb_day', 'Area', 'ltfs_median_tenor', 'disb_mon', 'Tenure', 'next_correctedDISBURSED-AMT/HIGH CREDIT_value', 'prev_tenor_value', 'mat_year', 'total_sanctioned_amount_Personal Loan', 'mean_correctedDISBURSED-AMT/HIGH CREDIT', 'sum_correctedDISBURSED-AMT/HIGH CREDIT', 'mat_mon', 'std_count', 'prev_correctedDISBURSED-AMT/HIGH CREDIT_value', '1_yr_frm_disbmax_day_start_day_diff_app_vs_other', 'std_percent_paid_off', 'min_correctedDISBURSED-AMT/HIGH CREDIT', 'gold_4_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', 'mean_percent_paid_off', 'next_correctedCURRENT-BAL_value', 'ltfs_std_count', 'mat_dow', 'std_correctedDISBURSED-AMT/HIGH CREDIT', '2_yr_frm_disbmax_day_start_day_diff_app_vs_other', 'ltfs_min_correctedDISBURSED-AMT/HIGH CREDIT', 'ManufacturerID', 'total_count', 'min_percent_paid_off', 'gold_2_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', 'max_days_bw_loans', 'total_sanctioned_amount__between_0_and_365_days', 'min_day_start_day_diff_app_vs_other', 'total_sanctioned_amount_Tractor Loan', 'mean_correctedCURRENT-BAL', 'overall_percent_paid_off', 'max_correctedDISBURSED-AMT/HIGH CREDIT', 'mean_days_bw_loans', 'overall_percentage_paid_off__between_0_and_365_days', 'disb_dow', 'min_days_bw_loans', 'LoanStatus', 'AmountFinance', 'gold_1_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', 'sum_correctedCURRENT-BAL', 'ltfs_min_percent_paid_off', 'std_correctedCURRENT-BAL', 'overall_percentage_paid_off_Tractor Loan', 'std_day_start_day_diff_app_vs_other', 'overall_percentage_paid_off_Personal Loan', 'std_days_bw_loans', '1_yr_frm_disbmin_day_start_day_diff_app_vs_other', 'gold_3_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', '0_yr_frm_disbmax_day_start_day_diff_app_vs_other', '1_yr_frm_disbmean_day_start_day_diff_app_vs_other', '1_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', 'mean_day_start_day_diff_app_vs_other', '2_yr_frm_disbmin_day_start_day_diff_app_vs_other', 'ltfs_mean_correctedDISBURSED-AMT/HIGH CREDIT', '2_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', 'ltfs_sum_correctedDISBURSED-AMT/HIGH CREDIT', 'PaymentMode', 'max_percent_paid_off', 'min_correctedCURRENT-BAL', 'max_correctedCURRENT-BAL', 'next_tenor_value', 'mean_dpd_str', '1_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', 'DisbursalAmount', 'ltfs_total_count', 'total_curr_bal__between_0_and_365_days', 'ltfs_total_sanctioned_amount_Tractor Loan', 'total_curr_bal_Tractor Loan', '1_yr_frm_disbmin_tenor', 'xxx_count', '4_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', '2_yr_frm_disbmean_day_start_day_diff_app_vs_other', 'ltfs_max_percent_paid_off', '2_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', 'ltfs_max_correctedCURRENT-BAL', 'ltfs_overall_percentage_paid_off__between_0_and_365_days', 'ltfs_min_correctedCURRENT-BAL', 'max_day_start_day_diff_app_vs_other', '0_yr_frm_disbmin_day_start_day_diff_app_vs_other', 'std_dpd_str', 'ltfs_min_day_start_day_diff_app_vs_other', 'ltfs_total_sanctioned_amount__between_0_and_365_days', 'ltfs_mean_percent_paid_off', 'ltfs_total_curr_bal__between_0_and_365_days', '1_yr_frm_disbmedian_tenor', '3_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', 'gold_1_yr_frm_disbmax_day_start_day_diff_app_vs_other', 'ltfs_max_correctedDISBURSED-AMT/HIGH CREDIT', '1_yr_frm_disbmin_time_delta_anchor_close_vs_start', '2_yr_frm_disbmin_tenor', '0_yr_frm_disbmax_tenor', '1_yr_frm_disbmax_tenor', '0_yr_frm_disbmin_tenor', 'total_sanctioned_amount_Overdraft', '0_yr_frm_disbmedian_tenor', '1_yr_frm_disbmax_time_delta_anchor_close_vs_start', 'ltfs_max_day_start_day_diff_app_vs_other', 'max_dpd_str', 'ddd_count', '0_yr_frm_disbmin_time_delta_anchor_close_vs_start', '4_yr_frm_disbmin_tenor', '3_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', '1_yr_frm_disbmin_disb_ratio', 'num_open_accounts', '1_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', 'total_sanctioned_amount_Gold Loan', '0_yr_frm_disbmean_day_start_day_diff_app_vs_other', 'ltfs_mean_correctedCURRENT-BAL', 'overall_percentage_paid_off_Business Loan Priority Sector  Agriculture', '3_yr_frm_disbmax_day_start_day_diff_app_vs_other', '0_yr_frm_disbmax_time_delta_anchor_close_vs_start', 'mat_day', '4_yr_frm_disbtotal_sanctioned_amount_Overdraft', 'prev_correctedCURRENT-BAL_value', 'sum_days_bw_loans', 'ltfs_overall_percentage_paid_off_Tractor Loan', 'ltfs_num_open_accounts', '0_yr_frm_disbmin_disb_ratio', 'ltfs_num_closed_accounts', '1_yr_frm_disbsum_disb_ratio', '1_yr_frm_disbmean_time_delta_anchor_close_vs_start', 'ltfs_mean_days_bw_loans', '0_yr_frm_disbmean_time_delta_anchor_close_vs_start', '1_yr_frm_disbtotal_sanctioned_amount_Overdraft', 'late_count', '4_yr_frm_disbstd_percent_paid_off', '0_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', 'ltfs_mean_day_start_day_diff_app_vs_other', '4_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', 'total_sanctioned_amount__between_365_and_730_days', 'std_correctedOVERDUE-AMT', 'ltfs_sum_correctedCURRENT-BAL', '1_yr_frm_disbmax_disb_ratio', '0_yr_frm_disbsum_disb_ratio', '2_yr_frm_disbtotal_sanctioned_amount_Overdraft', 'gold_2_yr_frm_disbmax_day_start_day_diff_app_vs_other', '2_yr_frm_disbmin_disb_ratio', 'ltfs_min_days_bw_loans', 'ltfs_max_days_bw_loans', '4_yr_frm_disbmin_percent_paid_off', '1_yr_frm_disbmean_disb_ratio', '0_yr_frm_disbsum_correctedDISBURSED-AMT/HIGH CREDIT', '2_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', 'ltfs_overall_percent_paid_off', '2_yr_frm_disbmax_time_delta_anchor_close_vs_start', 'ltfs_std_correctedDISBURSED-AMT/HIGH CREDIT', 'ltfs_std_correctedCURRENT-BAL', '2_yr_frm_disbmax_tenor', '3_yr_frm_disbmin_tenor', 'overall_percentage_paid_off_Gold Loan', '1_yr_frm_disbsum_correctedDISBURSED-AMT/HIGH CREDIT', 'gold_4_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', 'gold_1_yr_frm_disbmean_day_start_day_diff_app_vs_other', 'mean_correctedOVERDUE-AMT', '4_yr_frm_disbstd_correctedDISBURSED-AMT/HIGH CREDIT', 'overall_percentage_paid_off_Kisan Credit Card', 'gold_1_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', 'ltfs_mean_dpd_str', 'overall_percentage_paid_off_Overdraft', '4_yr_frm_disbmedian_tenor', '4_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', '4_yr_frm_disbmax_tenor', 'ltfs_std_percent_paid_off', '2_yr_frm_disbmin_time_delta_anchor_close_vs_start', '2_yr_frm_disbmedian_tenor', '3_yr_frm_disbmedian_tenor', 'ltfs_total_curr_bal_Tractor Loan', 'ltfs_std_dpd_str', '1_yr_frm_disbmin_percent_paid_off', '3_yr_frm_disbmin_disb_ratio', 'trac_4_yr_frm_disbmin_tenor_is_ltfs', '0_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', '0_yr_frm_disbmean_disb_ratio', '4_yr_frm_disbmean_day_start_day_diff_app_vs_other', 'num_accounts_Personal Loan', '4_yr_frm_disboverall_percent_paid_off', '1_yr_frm_disbstd_percent_paid_off', 'mean_dpd_str_application_loan', '1_yr_frm_disbmin_correctedCURRENT-BAL', 'gold_1_yr_frm_disbmin_day_start_day_diff_app_vs_other', '1_yr_frm_disbstd_days_bw_loans', '0_yr_frm_disbmax_disb_ratio', 'total_sanctioned_amount_Business Loan Priority Sector  Agriculture', '4_yr_frm_disbmax_day_start_day_diff_app_vs_other', '2_yr_frm_disbmean_disb_ratio', 'trac_2_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', '3_yr_frm_disbmax_tenor', 'trac_2_yr_frm_disbmax_time_delta_anchor_close_vs_start', 'overall_percentage_paid_off__between_365_and_730_days', 'trac_1_yr_frm_disbmax_day_start_day_diff_app_vs_other', '4_yr_frm_disbmin_disb_ratio', 'gold_2_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', 'trac_2_yr_frm_disbmax_day_start_day_diff_app_vs_other', '0_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', '2_yr_frm_disbmin_correctedCURRENT-BAL', 'trac_1_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', 'gold_4_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', '2_yr_frm_disbsum_disb_ratio', 'overall_percentage_paid_off_Auto Loan (Personal)', '4_yr_frm_disbsum_correctedCURRENT-BAL', 'trac_4_yr_frm_disbmax_tenor_is_ltfs', '4_yr_frm_disbstd_correctedCURRENT-BAL', '2_yr_frm_disbmax_disb_ratio', '4_yr_frm_disbmin_day_start_day_diff_app_vs_other', 'trac_2_yr_frm_disbmin_time_delta_anchor_close_vs_start', '2_yr_frm_disbsum_correctedDISBURSED-AMT/HIGH CREDIT', '3_yr_frm_disbmean_day_start_day_diff_app_vs_other', '4_yr_frm_disbnum_open_accounts', '1_yr_frm_disbmin_days_bw_loans', '2_yr_frm_disbmean_time_delta_anchor_close_vs_start', 'ltfs_max_dpd_str', 'Frequency', '1_yr_frm_disbmean_percent_paid_off', '0_yr_frm_disbmin_percent_paid_off', '4_yr_frm_disbmin_tenor_is_ltfs', 'max_dpd_str_application_loan', '1_yr_frm_disbmean_correctedCURRENT-BAL', '0_yr_frm_disbstd_disb_ratio', 'std_dpd_str_application_loan', '0_yr_frm_disboverall_percent_paid_off', '_90_count_application_loan', '1_yr_frm_disbstd_correctedDISBURSED-AMT/HIGH CREDIT', '2_yr_frm_disbstd_percent_paid_off', '2_yr_frm_disbstd_correctedDISBURSED-AMT/HIGH CREDIT', 'ltfs_max_dpd_str_application_loan', '2_yr_frm_disboverall_percentage_paid_off_Overdraft', '1_yr_frm_disbmax_percent_paid_off', '4_yr_frm_disbstd_disb_ratio', 'gold_3_yr_frm_disbmax_day_start_day_diff_app_vs_other', '3_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', 'max_correctedOVERDUE-AMT', 'overall_percentage_paid_off__between_730_and_1095_days', '3_yr_frm_disbmin_day_start_day_diff_app_vs_other', '1_yr_frm_disboverall_percentage_paid_off_Overdraft', '2_yr_frm_disbmin_percent_paid_off', '2_yr_frm_disbstd_disb_ratio', 'num_accounts_Overdraft', 'trac_2_yr_frm_disbmean_time_delta_anchor_close_vs_start', '3_yr_frm_disbsum_correctedDISBURSED-AMT/HIGH CREDIT', '1_yr_frm_disbstd_correctedCURRENT-BAL', '_30_count', 'gold_2_yr_frm_disbmin_day_start_day_diff_app_vs_other', 'gold_1_yr_frm_disbmax_tenor', 'trac_3_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', '1_yr_frm_disbstd_disb_ratio', '1_yr_frm_disbtotal_sanctioned_amount_Gold Loan', '4_yr_frm_disbmin_days_bw_loans', '0_yr_frm_disbstd_correctedDISBURSED-AMT/HIGH CREDIT', 'ltfs__90_count_application_loan', '3_yr_frm_disbmean_disb_ratio', 'total_sanctioned_amount__between_730_and_1095_days', '4_yr_frm_disbmax_correctedCURRENT-BAL', 'gold_0_yr_frm_disbmax_day_start_day_diff_app_vs_other', 'trac_4_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', 'trac_4_yr_frm_disbtotal_curr_bal_is_ltfs', '1_yr_frm_disbstd_time_delta_anchor_close_vs_start', 'curr_bal_grtr_0', '4_yr_frm_disbmean_disb_ratio', 'gold_1_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', '4_yr_frm_disbmean_correctedCURRENT-BAL', '0_yr_frm_disbmean_percent_paid_off', 'gold_2_yr_frm_disbmax_disb_ratio', '1_yr_frm_disbmax_correctedCURRENT-BAL', 'ltfs_std_dpd_str_application_loan', '1_yr_frm_disboverall_percent_paid_off', 'total_sanctioned_amount__between_1095_and_1460_days', 'gold_3_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', '4_yr_frm_disbmean_percent_paid_off', 'ltfs_mean_dpd_str_application_loan', 'ltfs_late_count', 'total_curr_bal_Gold Loan', '2_yr_frm_disbmean_correctedCURRENT-BAL', '4_yr_frm_disbstd_days_bw_loans', 'overall_percentage_paid_off_Other', '2_yr_frm_disbmax_correctedCURRENT-BAL', 'total_curr_bal_Kisan Credit Card', '4_yr_frm_disbmax_tenor_is_ltfs', 'trac_2_yr_frm_disbmin_disb_ratio', '2_yr_frm_disbmin_days_bw_loans', '3_yr_frm_disbmax_disb_ratio', '2_yr_frm_disbmax_days_bw_loans', 'total_curr_bal_Business Loan Priority Sector  Agriculture', 'trac_1_yr_frm_disbmin_time_delta_anchor_close_vs_start', 'SEX', 'gold_1_yr_frm_disbmax_disb_ratio', '2_yr_frm_disbmax_percent_paid_off', '4_yr_frm_disbnum_accounts_Overdraft', '1_yr_frm_disbmean_days_bw_loans', '2_yr_frm_disbtotal_sanctioned_amount_Gold Loan', 'gold_2_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', '3_yr_frm_disbsum_disb_ratio', 'std_count_application_loan', '3_yr_frm_disbstd_correctedDISBURSED-AMT/HIGH CREDIT', '3_yr_frm_disboverall_percentage_paid_off_Overdraft', 'num_closed_accounts', '0_yr_frm_disbsum_days_bw_loans', 'trac_2_yr_frm_disbmin_day_start_day_diff_app_vs_other', '4_yr_frm_disbsum_correctedDISBURSED-AMT/HIGH CREDIT', 'trac_2_yr_frm_disbtotal_sanctioned_amount_is_ltfs', '0_yr_frm_disbmin_days_bw_loans', '3_yr_frm_disbmin_percent_paid_off', 'gold_2_yr_frm_disbmean_day_start_day_diff_app_vs_other', 'total_curr_bal_Personal Loan', '3_yr_frm_disbtotal_sanctioned_amount_Overdraft', 'trac_4_yr_frm_disbmin_disb_ratio', '_60_count_application_loan', 'trac_4_yr_frm_disbtotal_sanctioned_amount_is_ltfs', '4_yr_frm_disbmax_disb_ratio', 'ltfs__60_count_application_loan', '2_yr_frm_disbstd_correctedCURRENT-BAL', 'total_curr_bal__between_365_and_730_days', '1_yr_frm_disbmax_days_bw_loans', '2_yr_frm_disbmean_percent_paid_off', 'gold_1_yr_frm_disbmin_tenor', '1_yr_frm_disbstd_day_start_day_diff_app_vs_other', 'total_sanctioned_amount_Auto Loan (Personal)', '4_yr_frm_disbindividual_accounts', '1_yr_frm_disbmax_tenor_is_ltfs', '1_yr_frm_disbsum_days_bw_loans', '0_yr_frm_disbmean_correctedCURRENT-BAL', '0_yr_frm_disbmean_days_bw_loans', '_60_count', '0_yr_frm_disbstd_percent_paid_off', '2_yr_frm_disbmean_days_bw_loans', 'gold_1_yr_frm_disbmedian_tenor', '4_yr_frm_disbsum_days_bw_loans', '3_yr_frm_disbmin_days_bw_loans', '4_yr_frm_disbsum_disb_ratio', 'gold_2_yr_frm_disbmin_percent_paid_off', '4_yr_frm_disbmean_days_bw_loans', '2_yr_frm_disbstd_time_delta_anchor_close_vs_start', 'ltfs_ddd_count', 'individual_accounts', '4_yr_frm_disbmax_percent_paid_off', '0_yr_frm_disbsum_correctedCURRENT-BAL', 'trac_2_yr_frm_disbmean_day_start_day_diff_app_vs_other', '1_yr_frm_disbsum_correctedCURRENT-BAL', '3_yr_frm_disbstd_days_bw_loans', 'ltfs__30_count', 'sum_correctedOVERDUE-AMT', '3_yr_frm_disbtotal_sanctioned_amount_Gold Loan', 'trac_2_yr_frm_disbmean_disb_ratio', 'overall_percentage_paid_off__between_1460_and_3650_days', 'trac_1_yr_frm_disbmax_time_delta_anchor_close_vs_start', 'ltfs_sum_days_bw_loans', 'gold_0_yr_frm_disbmean_day_start_day_diff_app_vs_other', 'trac_2_yr_frm_disbmin_tenor', 'gold_1_yr_frm_disbmin_percent_paid_off', 'trac_2_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', 'trac_1_yr_frm_disbsum_disb_ratio', '4_yr_frm_disbnum_accounts', 'gold_1_yr_frm_disbmean_disb_ratio', '2_yr_frm_disboverall_percent_paid_off', '1_yr_frm_disbmean_correctedOVERDUE-AMT', 'trac_1_yr_frm_disbsum_correctedDISBURSED-AMT/HIGH CREDIT', '2_yr_frm_disbsum_days_bw_loans', 'gold_1_yr_frm_disbsum_disb_ratio', 'total_sanctioned_amount_Kisan Credit Card', 'trac_2_yr_frm_disboverall_percentage_paid_off_is_ltfs', 'gold_1_yr_frm_disbmin_time_delta_anchor_close_vs_start', 'trac_1_yr_frm_disbmean_time_delta_anchor_close_vs_start', '0_yr_frm_disbmax_percent_paid_off', 'gold_0_yr_frm_disbmin_tenor', '3_yr_frm_disbstd_day_start_day_diff_app_vs_other', '0_yr_frm_disbmin_correctedCURRENT-BAL', 'trac_1_yr_frm_disbtotal_sanctioned_amount_is_ltfs', '0_yr_frm_disbstd_correctedCURRENT-BAL', '0_yr_frm_disbmax_days_bw_loans', 'gold_0_yr_frm_disbmin_day_start_day_diff_app_vs_other', 'total_curr_bal_Auto Loan (Personal)', '4_yr_frm_disbmin_correctedCURRENT-BAL', 'trac_4_yr_frm_disbsum_disb_ratio', 'trac_2_yr_frm_disbmax_disb_ratio', '3_yr_frm_disbstd_percent_paid_off', 'overall_percentage_paid_off__between_1095_and_1460_days', 'trac_4_yr_frm_disbmax_day_start_day_diff_app_vs_other', 'trac_4_yr_frm_disbmin_day_start_day_diff_app_vs_other', '3_yr_frm_disbindividual_accounts', 'trac_4_yr_frm_disbmax_disb_ratio', '0_yr_frm_disbtotal_sanctioned_amount_Gold Loan', 'total_curr_bal__between_730_and_1095_days', '4_yr_frm_disbcurr_bal_grtr_0', '3_yr_frm_disbmean_days_bw_loans', '0_yr_frm_disbmin_correctedOVERDUE-AMT', 'gold_2_yr_frm_disbmin_tenor', '4_yr_frm_disboverall_percentage_paid_off_Overdraft', 'gold_1_yr_frm_disbmax_time_delta_anchor_close_vs_start', 'trac_4_yr_frm_disbmedian_tenor', 'trac_2_yr_frm_disbmin_tenor_is_ltfs', 'ltfs_total_sanctioned_amount__between_1095_and_1460_days', 'trac_4_yr_frm_disboverall_percentage_paid_off_is_ltfs', '3_yr_frm_disbstd_disb_ratio', '2_yr_frm_disbsum_correctedCURRENT-BAL', 'gold_0_yr_frm_disbsum_disb_ratio', 'trac_4_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', '0_yr_frm_disbmax_correctedCURRENT-BAL', '3_yr_frm_disbmean_correctedCURRENT-BAL', 'total_sanctioned_amount__between_1460_and_3650_days', '2_yr_frm_disbtotal_sanctioned_amount_Tractor Loan', '1_yr_frm_disbmin_tenor_is_ltfs', '0_yr_frm_disbstd_day_start_day_diff_app_vs_other', 'trac_4_yr_frm_disbmean_disb_ratio', '2_yr_frm_disbstd_day_start_day_diff_app_vs_other', 'gold_0_yr_frm_disbmax_time_delta_anchor_close_vs_start', '2_yr_frm_disbmean_correctedOVERDUE-AMT', '4_yr_frm_disbstd_day_start_day_diff_app_vs_other', 'trac_2_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', 'trac_1_yr_frm_disbmax_tenor_is_ltfs', 'trac_1_yr_frm_disbmean_day_start_day_diff_app_vs_other', 'trac_4_yr_frm_disbmin_tenor', 'trac_3_yr_frm_disbmax_tenor_is_ltfs', 'trac_3_yr_frm_disbmin_tenor', 'trac_2_yr_frm_disbtotal_curr_bal_is_ltfs', 'gold_0_yr_frm_disbmedian_tenor', '4_yr_frm_disbtotal_curr_bal_Overdraft', '1_yr_frm_disbtotal_sanctioned_amount_Tractor Loan', 'gold_1_yr_frm_disbmean_time_delta_anchor_close_vs_start', 'gold_0_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', 'gold_0_yr_frm_disbmin_time_delta_anchor_close_vs_start', 'gold_2_yr_frm_disbmax_tenor', 'ltfs_total_sanctioned_amount__between_730_and_1095_days', '1_yr_frm_disbmin_correctedOVERDUE-AMT', 'gold_2_yr_frm_disbmean_disb_ratio', '2_yr_frm_disbtotal_sanctioned_amount_is_ltfs', 'gold_0_yr_frm_disbmax_tenor', 'total_sanctioned_amount_Other', 'gold_0_yr_frm_disbmax_disb_ratio', 'trac_4_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', 'ltfs_total_curr_bal__between_365_and_730_days', '4_yr_frm_disbmax_days_bw_loans', 'gold_1_yr_frm_disbmax_correctedCURRENT-BAL', 'gold_3_yr_frm_disbmax_disb_ratio', 'trac_3_yr_frm_disbtotal_sanctioned_amount_is_ltfs', 'gold_1_yr_frm_disbmin_disb_ratio', 'num_accounts', 'ltfs_std_day_start_day_diff_app_vs_other', '4_yr_frm_disbtotal_sanctioned_amount_Tractor Loan', 'ltfs_joint_accounts', 'gold_0_yr_frm_disbmax_correctedDISBURSED-AMT/HIGH CREDIT', '4_yr_frm_disbtotal_sanctioned_amount_is_ltfs', '3_yr_frm_disbmin_time_delta_anchor_close_vs_start', 'trac_3_yr_frm_disbmax_day_start_day_diff_app_vs_other', '3_yr_frm_disbmean_percent_paid_off', 'gold_3_yr_frm_disbmin_day_start_day_diff_app_vs_other', 'gold_4_yr_frm_disbmax_disb_ratio', '3_yr_frm_disbsum_days_bw_loans', '2_yr_frm_disbstd_days_bw_loans', 'gold_2_yr_frm_disbsum_disb_ratio', '1_yr_frm_disbmax_correctedOVERDUE-AMT', '3_yr_frm_disbmax_correctedCURRENT-BAL', 'trac_2_yr_frm_disbmedian_tenor', '3_yr_frm_disbtotal_sanctioned_amount_Tractor Loan', 'gold_3_yr_frm_disbmean_disb_ratio', 'gold_0_yr_frm_disbmean_disb_ratio', '3_yr_frm_disbsum_correctedCURRENT-BAL', 'gold_3_yr_frm_disbmin_correctedDISBURSED-AMT/HIGH CREDIT', '0_yr_frm_disbindividual_accounts', 'gold_1_yr_frm_disbsum_correctedDISBURSED-AMT/HIGH CREDIT', 'gold_3_yr_frm_disbsum_disb_ratio', '2_yr_frm_disbmax_tenor_is_ltfs', 'ltfs_total_curr_bal__between_730_and_1095_days', '3_yr_frm_disbstd_correctedCURRENT-BAL', 'trac_4_yr_frm_disbmax_tenor', 'ltfs_total_sanctioned_amount__between_365_and_730_days', '1_yr_frm_disbstd_correctedOVERDUE-AMT', 'late_count_application_loan', '_30_count_application_loan', '4_yr_frm_disbmean_correctedOVERDUE-AMT', 'gold_0_yr_frm_disbmin_disb_ratio', 'gold_1_yr_frm_disbstd_correctedDISBURSED-AMT/HIGH CREDIT', 'trac_1_yr_frm_disbmean_correctedDISBURSED-AMT/HIGH CREDIT', 'trac_1_yr_frm_disbmin_tenor', '4_yr_frm_disbstd_correctedOVERDUE-AMT', 'num_accounts_Gold Loan', 'gold_2_yr_frm_disbsum_correctedDISBURSED-AMT/HIGH CREDIT']

In [19]:
len(use_cols)

1549

In [29]:
# df[['ID']+use_cols].to_pickle('sahil_top_500_feats.pkl')

In [20]:
import gc
gc.collect()

44

In [21]:
folds = StratifiedKFold(5, shuffle = True, random_state = 2)
folds = [(x,y) for x,y in folds.split(train,train[target])]

In [22]:
len(use_cols)

1549

In [23]:
lgbm_space = {
    "n_estimators": 5000,
    "num_leaves": hp.quniform("num_leaves", 32, 192, 32),
    "min_child_weight": hp.quniform("min_child_weight", 10, 100, 20),
    "subsample": hp.quniform("subsample", 0.5, 1, 0.1),
    "colsample_bytree": hp.quniform("colsample_bytree", 0.5, 1, 0.1),
    "subsample_freq": 5,
    "objective": "multiclass",
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "n_jobs": -1,
}

In [24]:
est = Estimator(model = LGBMClassifier(),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [25]:
%%time
hyp= HyperOptModelSelection(model=est,space=lgbm_space,max_evals=50,is_maximize=True,log_file_path='hyp4.txt')

CPU times: user 202 µs, sys: 191 µs, total: 393 µs
Wall time: 303 µs


In [26]:
%%time
# hyp.fit(train[use_cols].values,train[target].values)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.29 µs


In [27]:
# params={'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'min_child_weight': 40.0, 'n_estimators': 5000,
#         'n_jobs': -1, 'num_leaves': 96, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5}

params = {'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'min_child_weight': 20.0, 'n_estimators': 5000,
 'n_jobs': -1, 'num_leaves': 64, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5}

In [28]:
# est = Estimator(model = LGBMClassifier(colsample_bytree=0.7000000000000001, min_child_weight=60.0,learning_rate=0.1,
#                n_estimators=5000, num_leaves=64, objective='multiclass',metric='None',
#                subsample_freq=5),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

est = Estimator(model = LGBMClassifier(**params),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [29]:
temp = est.fit_transform(train[use_cols].values,train[target].values)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.258375	valid_1's multi_logloss: 0.132077
Early stopping, best iteration is:
[99]	valid_0's multi_logloss: 0.258368	valid_1's multi_logloss: 0.132906
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.255681	valid_1's multi_logloss: 0.133248
Early stopping, best iteration is:
[91]	valid_0's multi_logloss: 0.255486	valid_1's multi_logloss: 0.141035
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.256969	valid_1's multi_logloss: 0.131944
Early stopping, best iteration is:
[95]	valid_0's multi_logloss: 0.256868	valid_1's multi_logloss: 0.136149
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.257432	valid_1's multi_logloss: 0.132902
[200]	valid_0's multi_logloss: 0.263187	valid_1's multi_logloss: 0.0735875
Early stopping, best iteration is:
[100]	valid_0's multi_

In [30]:
est.cv_scores,est.overall_cv_score

([0.7334718744600853,
  0.7304890392602986,
  0.7374923985736554,
  0.7337399913717115,
  0.7442295329769361],
 0.7360326951257737)

In [31]:
feat_imp = est.feature_importance_df(use_cols)

In [33]:
feat_imp.head(100)

,column,feature_importance,rank
0,MonthlyIncome,0.013520,1
521,MonthlyIncome_freq,0.012352,2
766,DisbursalDate_days_since_start,0.009206,3
3,emi_sal_ratio,0.008461,4
785,ZiPCODEdd_min,0.007785,5
501,ZiPCODE,0.007681,6
512,EMI_freq,0.007508,7
4,AGE,0.007490,8
779,SupplierIDdd_min,0.007425,9
1,cal_tenor,0.007308,10


In [34]:
f1_score(train[target],pd.Series(temp.argmax(axis=1)),average='macro')

0.7360326951257737

In [35]:
test_preds = [est.predict_proba(test[use_cols].values) for est in est.fitted_models]

In [36]:
test['Top-up Month'] = np.sum(test_preds,axis=0).argmax(axis=1)

In [37]:
test['Top-up Month'].value_counts(1)

0    0.920109
6    0.036758
5    0.014852
2    0.007799
4    0.007664
3    0.007189
1    0.005629
Name: Top-up Month, dtype: float64

In [39]:
sub = test[['ID','Top-up Month']]

In [40]:
sub['Top-up Month'] = sub['Top-up Month'].map(reverse_map)

In [41]:
sub.to_csv('NM_SV_LGBM_CV0.736.csv',index=False)